# Creating plots

## 0) Imports and definitions

In [1]:
# Standard
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

# Viz
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Other
import zeep
import json
import urllib
import requests
from unidecode import unidecode

## 1) WSDL

In [2]:
wsdl = './FachadaWSSGS.wsdl'
client = zeep.Client(wsdl=wsdl)

In [3]:
long_array_type = client.get_type('ns0:ArrayOfflong')
r = client.service.getValoresSeriesVO(long_array_type([7384]), '01/01/2000', '05/04/2021')
aux = zeep.helpers.serialize_object(r[0]['valores'])

In [4]:
df_vendas = pd.DataFrame(aux)
df_vendas['valor'] = df_vendas['valor'].apply(lambda x: int(x['_value_1']))

In [6]:
df_vendas = df_vendas.rename(columns={'valor': 'vendas'})
df_vendas = df_vendas[['mes', 'ano', 'vendas']]
df_vendas.tail()

,mes,ano,vendas
251,12,2020,194668
252,1,2021,130794
253,2,2021,128091
254,3,2021,141928
255,4,2021,127444


## 3) Data POA 

In [2]:
start_year, start_month = 2021, 1
end_year, end_month = 2021, 2
next_month_date = date(end_year, end_month, 1) + timedelta(31)
no_inclusive_end_year, no_inclusive_end_month = next_month_date.year, next_month_date.month

In [3]:
# url = 'https://dadosabertos.poa.br/api/3/action/datastore_search?resource_id=b56f8123-716a-4893-9348-23945f1ea1b9&limit=100000'
sql_query = f"""sql=SELECT * from "b56f8123-716a-4893-9348-23945f1ea1b9" WHERE data >= '{start_year}-{start_month}-01' and data < '{no_inclusive_end_year}-{no_inclusive_end_month}-01' """
url = f"""https://dadosabertos.poa.br/api/3/action/datastore_search_sql?{sql_query}"""
r = requests.get(url)
data_json = r.json()

ConnectionError: HTTPSConnectionPool(host='dadosabertos.poa.br', port=443): Max retries exceeded with url: /api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22b56f8123-716a-4893-9348-23945f1ea1b9%22%20WHERE%20data%20%3E=%20'2021-1-01'%20and%20data%20%3C%20'2021-3-01'%20 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F308EF4970>: Failed to establish a new connection: [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu'))

In [ ]:
df_acidentes = pd.DataFrame.from_dict(data_json['result']['records'])

In [17]:
df_acidentes['dia'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x, errors='coerce').day)
df_acidentes['mes'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x, errors='coerce').month)
df_acidentes['ano'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x, errors='coerce').year)
df_acidentes['tipo_acid'] = df_acidentes['tipo_acid'].apply(lambda x: unidecode(x.lower()))

In [18]:
df_acidentes = df_acidentes[['dia', 'mes', 'ano', 'tipo_acid', 'feridos', 'fatais', 'caminhao', 'moto', 'latitude', 'longitude']]
df_acidentes.head()

,dia,mes,ano,tipo_acid,feridos,fatais,caminhao,moto,latitude,longitude
0,17,1,2021,abalroamento,1,0,0,0,None,None
1,9,1,2021,choque,1,0,0,1,None,None
2,25,1,2021,atropelamento,1,0,0,0,-30.03655189375627,-51.18669270222279
3,26,1,2021,abalroamento,1,0,0,0,-30.005594603794567,-51.19952917099
4,26,1,2021,colisao,0,0,0,0,-29.998958341748633,-51.19813442230225


## 4) Auxiliary functions

In [ ]:
def plot_acidentes_with_filters(df, year, cat_values=[-1, -1, -1, -1]):

    cat_list = ['feridos', 'fatais', 'caminhao', 'moto']

    df_aux = df[(df['ano'] == year)]

    for col, value, in zip(cat_list, cat_values):
        if value > 0:
            df_aux = df_aux[df_aux[col] == value]
    
    df_aux = df_aux.sort_values(by='mes')
    fig = px.histogram(df_aux,
                    x="mes",
                    barmode='group',
                    width=800,
                    height=400, 
                    labels={'2016': ' '},
                    title="Acidentes por mês")
    fig.update_layout(showlegend=False, bargap=0.2)
    fig.show()

In [ ]:
def plot_acidentes_per_month(df, year, month, tipo_acid=False):

    df_aux = df[(df['mes'] == month) & (df['ano'] == year)]
    if not tipo_acid:
        fig = px.histogram(df_aux, 
                           x="dia", 
                           barmode='group', 
                           width=800,
                           height=400, 
                           labels={'dia': 'Dia do mês'},
                           title="Acidentes por dia")
        fig.update_layout(showlegend=False, bargap=0.2)
    else:
        fig = px.histogram(df_aux,
                           x="mes", 
                           color='tipo_acid',
                           barmode='group',
                           width=800,
                           height=400, 
                           labels={'mes': 'Mês'},
                           title="Acidentes por mês")
        fig.update_layout(showlegend=True, bargap=0.2)
    fig.show()

In [ ]:
def plot_vendas_per_month(df, year):

    df_aux = df[(df['ano'] == year)]

    fig = px.bar(df_aux,
                x="mes", 
                y='vendas',
                width=800,
                height=400,
                labels={'mes': 'Mês',
                        'vendas': 'Número de vendas'},
                title="Vendas por mês")
    fig.update_layout(showlegend=False, bargap=0.2)
    fig.show()

In [ ]:
def plot_acidentes_versus_vendas(df_acidentes, df_vendas, year):

    df_acid_aux = df_acidentes[df_acidentes['ano'] == year]
    df_vend_aux = df_vendas[df_vendas['ano'] == year]

    df_acid_aux = df_acid_aux[['mes', 'ano']].groupby(by='mes').count().reset_index().rename(columns={'ano': 'acidentes'})
    df_vend_aux = df_vend_aux.drop(columns=['ano']).reset_index(drop=True)

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(x=df_vend_aux['mes'], y=df_vend_aux['vendas'], name="Vendas"),
        secondary_y=True,
    )

    # Add traces
    fig.add_trace(
        go.Scatter(x=df_acid_aux['mes'], y=df_acid_aux['acidentes'], name="Acidentes"),
        secondary_y=False,
    )


    # Add figure title
    fig.update_layout(
        title_text="Número de vendas versus acidentes"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Mês")

    # Set y-axes titles
    fig.update_yaxes(title_text="Número de acidentes", secondary_y=False)
    fig.update_yaxes(title_text="Número de vendas", secondary_y=True)

    fig.show()

## 5) Plots

In [11]:
plot_vendas_per_month(df_vendas, 2018)

NameError: name 'plot_vendas_per_month' is not defined

In [ ]:
plot_acidentes_with_filters(df_acidentes, 2016, [1, -1, -1, -1])

In [ ]:
plot_acidentes_per_month(df_acidentes, 2017, 12, tipo_acid=True)

In [ ]:
plot_acidentes_per_month(df_acidentes, 2017, 10)


In [ ]:
plot_acidentes_versus_vendas(df_acidentes, df_vendas, 2017)

In [9]:
pd.read_csv('csi_project/static/py/cat_acidentes.csv', sep=';')['data']

0        2020-10-17 00:00:00
1        2016-01-01 00:00:00
2        2016-01-02 00:00:00
3        2016-01-02 00:00:00
4        2016-01-02 00:00:00
                ...         
68708    2021-08-27 00:00:00
68709    2021-08-27 00:00:00
68710    2021-08-27 00:00:00
68711    2021-08-27 00:00:00
68712    2021-08-30 00:00:00
Name: data, Length: 68713, dtype: object

In [10]:
pd.to_datetime(pd.read_csv('csi_project/static/py/cat_acidentes.csv', sep=';')['data'], format= '%Y-%m-%d 00:00:00', errors='coerce')

0       2020-10-17
1       2016-01-01
2       2016-01-02
3       2016-01-02
4       2016-01-02
           ...    
68708   2021-08-27
68709   2021-08-27
68710   2021-08-27
68711   2021-08-27
68712   2021-08-30
Name: data, Length: 68713, dtype: datetime64[ns]